# Simple simulator to demonstrate relationship between DEM error and geolocation error

This notebook is a simple demonstration of Jupyter's interactive capabilities. We simulate a multiple SAR imaging scenario that includes:

1. Three SAR imaging platforms flying at altitudes of roughly 12-14 km, that can be repositioned by the user. Two of these platforms are observing a target of interest from one side (ascending / right looking) and one of the platforms is observing the target from the opposite side (descending / left looking).
2. The platforms are imaging a single target on the ground which is actually at an altitude of 500m but whose exact elevation is not well known when processing the data.


We use simple 2D cartesian system to compute range differences and geometry characteristics to explain effect of DEM errors in geolocating the data; and its implications for change detection using data acquired from the same imaging geometries versus diverse imaging geometries.


In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.lines import Line2D
import numpy as np
from scipy.interpolate import interp1d

##Simple class to create a draggable point that the user can play with
class DraggablePoint:

    # http://stackoverflow.com/questions/21654008/matplotlib-drag-overlapping-points-interactively

    lock = None #  only one can be animated at a time

    def __init__(self, parent, x=0.1, y=0.1, size=500, color='r'):

        self.parent = parent
        self.point = patches.Ellipse((x, y), size, size, fc=color, alpha=0.5, edgecolor=color)
        self.x = x
        self.y = y
        parent.fig.axes[0].add_patch(self.point)
        self.press = None
        self.background = None
        self.connect()

    def connect(self):

        'connect to all the events we need'

        self.cidpress = self.point.figure.canvas.mpl_connect('button_press_event', self.on_press)
        self.cidrelease = self.point.figure.canvas.mpl_connect('button_release_event', self.on_release)
        self.cidmotion = self.point.figure.canvas.mpl_connect('motion_notify_event', self.on_motion)


    def on_press(self, event):

        if event.inaxes != self.point.axes:
            return
        
        if DraggablePoint.lock is not None:
            return
        
        contains, attrd = self.point.contains(event)
        if not contains:
            return
        
        self.press = (self.point.center), event.xdata, event.ydata
        DraggablePoint.lock = self

        # draw everything but the selected rectangle and store the pixel buffer
        canvas = self.point.figure.canvas
        axes = self.point.axes
        self.point.set_animated(True)

        canvas.draw()
        self.background = canvas.copy_from_bbox(self.point.axes.bbox)

        # now redraw just the rectangle
        axes.draw_artist(self.point)

        # and blit just the redrawn area
        canvas.blit(axes.bbox)
        #canvas.draw()


    def on_motion(self, event):

        if DraggablePoint.lock is not self:
            return
        if event.inaxes != self.point.axes: 
            return
        self.point.center, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        self.point.center = (self.point.center[0]+dx, self.point.center[1]+dy)

        canvas = self.point.figure.canvas
        axes = self.point.axes
        # restore the background region
        canvas.restore_region(self.background)

        # redraw just the current rectangle
        axes.draw_artist(self.point)

        self.x = self.point.center[0]
        self.y = self.point.center[1]

        # blit just the redrawn area
        canvas.blit(axes.bbox)
        #canvas.draw()


    def on_release(self, event):

        'on release we reset the press data'
        if DraggablePoint.lock is not self:
            return

        self.press = None
        DraggablePoint.lock = None

        # turn off the rect animation property and reset the background
        self.point.set_animated(False)

        self.background = None

        # redraw the full figure
        self.point.figure.canvas.draw()

        self.x = self.point.center[0]
        self.y = self.point.center[1]
        
        self.parent.updateFigure()

    def disconnect(self):

        'disconnect all the stored connection ids'

        self.point.figure.canvas.mpl_disconnect(self.cidpress)
        self.point.figure.canvas.mpl_disconnect(self.cidrelease)
        self.point.figure.canvas.mpl_disconnect(self.cidmotion)

In [ ]:
#Simple class for geometry computations and to update plots
class SARGeometry:
    '''
    Class to simulate an SAR imaging scenario from asc and desc geometries
    '''
    
    def __init__(self, width=10, height=6, demerr=10.0):
        '''
        Create figure and populate points on it.
        '''
        #Save wavelength
        self.demerr = demerr
        
        #Create figure
        self.fig = plt.figure(figsize=(width,height))
        
        #Create axes to include imaging targets and SAR positions
        self.imgrect = [0.1, 0.1, 0.8, 0.8]
        self.imgaxes = self.fig.add_axes(self.imgrect)
        
        #x-axis multiplier
        self.xscale = 80000.
        
        #Y-axis multiplier
        self.yscale = 15000.

        self.sar1text = None
        self.sar2text = None
        self.sar3text = None
        
        self.imgaxes.set_xlim([0, self.xscale])
        self.imgaxes.set_ylim([0, self.yscale])

        plt.show()
        
        #Add target points on ground
        self.target = None
        self.addTarget()
        
        #Add SAR antennas
        self.SARs = []
        self.addSARs()
               
        self.addText()
        self.updateFigure()
        
    def position2Canvas(self, x, y):
        '''
        Convert physical x,y to canvas position for plotting
        '''
        xpos = self.imgrect[0] + (x/self.xscale) * self.imgrect[2]
        ypos = self.imgrect[1] + (y/self.yscale) * self.imgrect[3]
        return xpos, ypos
    
    def canvas2Position(self, xpos, ypos):
        '''
        Convert canvas position to physical x,y
        '''
        x = (xpos - self.imgrect[0]) * (self.xscale / self.imgrect[2])
        y = (ypos - self.imgrect[1]) * (self.yscale / self.imgrect[3])
        return x, y
        
    def addTarget(self):
        '''
        We will add about 1 target at the center of the X-axis
        '''
        self.target = DraggablePoint(self, 0.5 * self.xscale, 500.)
            
            
    def addSARs(self):
        '''
        We will add 2 SAR platforms at top left and 1 at the topright
        '''
        self.SARs.append( DraggablePoint(self, 1000., 14000., color='b', size=1000))
        self.SARs.append( DraggablePoint(self, 3000., 13000., color='g', size=1000))
        self.SARs.append( DraggablePoint(self, self.xscale-1000., 13500., color='k', size=1000))
        self.imgaxes.set_ylabel('Y')
        self.imgaxes.set_xlabel('X')
        self.imgaxes.set_title('$\Delta h$ = {0}'.format(self.demerr))
    
    def addText(self, update=False):
        '''
        Create unw plot
        '''
        truePos = [self.target.x, self.target.y]
        errPos = [self.target.x, self.target.y + self.demerr]

        trueRng1 = np.linalg.norm([self.SARs[0].x-truePos[0], self.SARs[0].y - truePos[1]])
        trueRng2 = np.linalg.norm([self.SARs[1].x-truePos[0], self.SARs[1].y - truePos[1]])
        trueRng3 = np.linalg.norm([self.SARs[2].x-truePos[0], self.SARs[2].y - truePos[1]])
        
        falseX1 = np.sqrt(trueRng1**2 - (self.SARs[0].y-errPos[1])**2) + self.SARs[0].x
        falseX2 = np.sqrt(trueRng2**2 - (self.SARs[1].y-errPos[1])**2) + self.SARs[1].x
        falseX3 = -np.sqrt(trueRng3**2 - (self.SARs[2].y-errPos[1])**2) + self.SARs[2].x       
        
        if not update:
            self.sar1text = self.imgaxes.text(0.1*self.xscale,0.2*self.yscale, 
                                              'S1={0:.2f}'.format(truePos[0]-falseX1),
                                             color='b')
            self.sar2text = self.imgaxes.text(0.1*self.xscale,0.4*self.yscale,
                                              'S2={0:.2f}'.format(truePos[0]-falseX2),
                                             color='g')
            self.sar3text = self.imgaxes.text(0.75*self.xscale,0.3*self.yscale, 
                                              'S2={0:.2f}'.format(truePos[0]-falseX3),
                                             color='k')
        else:
            self.sar1text.set_text('S1={0:.2f}'.format(truePos[0]-falseX1))
            self.sar2text.set_text('S2={0:.2f}'.format(truePos[0]-falseX2))
            self.sar3text.set_text('S3={0:.2f}'.format(truePos[0]-falseX3))
    
    
    def updateFigure(self):
        '''
        Update the whole plot
        '''
        self.addText(update=True)
        self.fig.canvas.draw()
        

In [ ]:
insar = SARGeometry(demerr=10)

One is encouraged to interact with the plot and perform following experiments:


1. The plots display the geolocation error - i.e, wrong position in "X" of the target due to DEM Error. Notice the opposite signs on values for images that are acquired from different geometries.

2. Move the platforms around and observe relationship between incidence angle and magnitude of geolocation error. 

3. All the Blue and Green platforms as best as possible in line with the target (red) and notice that the magnitude of geolocation error is essentially the same. 
